# Machine Learning Project, Task 1: Feature Engineering

In [1]:
# Imports
import pandas as pd
import math
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from xgboost.sklearn import XGBRegressor
import xgboost as xgb

import osmnx as ox
from shapely.geometry import Point
gdf = ox.geocode_to_gdf('New York, NY, USA')
geom = gdf.loc[0, 'geometry']

# get the bounding box of the city
geom.bounds

(-74.25909, 40.477399, -73.7001809, 40.9161785)

In [2]:
df = pd.read_csv('data/cc_nyc_fare_train_tiny.csv', parse_dates=['pickup_datetime'], index_col=0)
#data = df_to_geojson(df, lat='pickup_latitude', lon='pickup_longitude')
df.head(2)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
key,,,,,,,
2010-02-03 20:51:29.0000003,12.9,2010-02-03 20:51:29+00:00,-73.954191,40.764029,-73.918043,40.766876,1
2013-06-09 13:42:00.00000036,14.5,2013-06-09 13:42:00+00:00,-74.004507,40.741932,-74.005212,40.705272,1


## Utility Methods

In [3]:
def haversine_distance(origin, destination):
    """
    # Formula to calculate the spherical distance between 2 coordinates, with each specified as a (lat, lng) tuple

    :param origin: (lat, lng)
    :type origin: tuple
    :param destination: (lat, lng)
    :type destination: tuple
    :return: haversine distance
    :rtype: float
    """
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(math.radians(lat1)) * math.cos(
        math.radians(lat2)) * math.sin(dlon / 2) * math.sin(dlon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

## Feature Engineering
> TODO: You need to implement the two functions below to take in raw data, perform data preprocessing and create new features using your knowledge of the problem domain. The main difference between processing test and training data is that **you cannot filter out records from the test set** (i.e., you have to return a prediction even if the input data may be an outlier).

> Feel free to add additional cells to explore the data. You will also find it very helpful to visualize the distribution of the dataset to get a sense of the trends or patterns. **However, you will want to exclude these cells when we export the notebook as an executable.** The submitter will exclude any cells tagged with `excluded_from_script`, so make sure you tag any cells containing exploration code appropriately. You can display the tags for each cell as such: `View > Cell Toolbar > Tags`.

In [4]:
def datetime_tomore(df):
    time_features = df.loc[:, ['pickup_datetime']]
    # TODO: extract time-related features from the `pickup_datetime` column.
    #       (replace "None" with your implementation)
    time_features['year'] = time_features.pickup_datetime.apply(lambda x:x.year)
    time_features['month'] = time_features.pickup_datetime.apply(lambda x:x.month)
    time_features['hour'] = time_features.pickup_datetime.apply(lambda x:x.hour)
    time_features['weekday'] = time_features.pickup_datetime.apply(lambda x:x.weekday)
    # quantize
    #time_features['hour_bin'] = pd.qcut(time_features['hour'],4).cat.codes
    return pd.concat([df, time_features], axis=1)

def filter_NYC(df):    
    # determine if a point is within the city boundary
    coord_list = list(df[['pickup_longitude','pickup_latitude']].to_records(index=False))
    return df[[geom.intersects(Point(coords)) for coords in coord_list]]

def process_distance(df):
    pick_up_loc = ["pickup_longitude","pickup_latitude"]
    drop_off_loc = ["dropoff_longitude","dropoff_latitude"]
    return df.apply(lambda x: haversine_distance((x[pick_up_loc]), (x[drop_off_loc])), axis=1)

def MSG_distance(df, mode="dropoff"):
    MSG_coor = (40.750298, -73.993324) # lat, lng
    MSG_lat, MSG_long = MSG_coor
    drop_off_loc = ["dropoff_latitude", "dropoff_longitude"]
    pick_up_loc = ["pickup_latitude","pickup_longitude"]
    if mode=="dropoff":
        return df.apply(lambda x: haversine_distance(MSG_coor, (x[drop_off_loc])), axis=1)
    else:
        return df.apply(lambda x: haversine_distance(MSG_coor, (x[pick_up_loc])), axis=1)

fare_high = df[['fare_amount']].quantile(0.999)[0]
fare_low = df[['fare_amount']].quantile(0.001)[0]
def drop_quantile(df):
    return df.loc[df.fare_amount > fare_low].loc[df.fare_amount < fare_high]

In [5]:
num_cols = ["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count"]
pred_col = ['fare_amount']

def process_train_data(raw_df):
    """
    TODO: Implement this method.
    
    You may drop rows if needed.

    :param raw_df: the DataFrame of the raw training data
    :return:  a DataFrame with the predictors created
    """
    #fil na
    raw_df[num_cols] = raw_df[num_cols].fillna(value=raw_df[num_cols].mean())
    # drop location
    raw_df = filter_NYC(raw_df)
    #raw_df[num_cols] = raw_df[num_cols].fillna(value=raw_df[num_cols].mean())
    # filter quantile
    raw_df = drop_quantile(raw_df)
    # datetime
    raw_df = datetime_tomore(raw_df)
    # distance
    raw_df["distance"] = process_distance(raw_df)
    #raw_df = raw_df.drop(num_cols[:-1], axis=1)
    return raw_df


def process_test_data(raw_df):
    """
    TODO: Implement this method.
    
    You should NOT drop any rows.

    :param raw_df: the DataFrame of the raw test data
    :return: a DataFrame with the predictors created
    """
    # fill mean
    raw_df[num_cols] = raw_df[num_cols].fillna(value=raw_df[num_cols].mean())
    # 
    # datetime
    raw_df = datetime_tomore(raw_df)
    # distance
    raw_df["distance"] = process_distance(raw_df)
    #raw_df = raw_df.drop(num_cols[:-1], axis=1)
    return raw_df

## Model Checking with XGBoost and k-fold Cross Validation
> As you iterate on your features, you want to quickly validate the model and evaluate if these new features help to improve your model's predictions. This process is known as model checking. You will use XGBoost to train the model, and use Root Mean Squared Error (RMSE) to quantify the performance. Cross validation is used to evaluate how the performance of the model will generalize to an unseen dataset.

In [6]:
# Load data
raw_train = pd.read_csv('data/cc_nyc_fare_train_tiny.csv', parse_dates=['pickup_datetime'])
print('Shape of the raw data: {}'.format(raw_train.shape))

Shape of the raw data: (10, 8)


In [7]:
# Determine threshold
fare_high = df[['fare_amount']].quantile(0.999)[0]
fare_low = df[['fare_amount']].quantile(0.001)[0]
def drop_quantile(df):
    return df.loc[df.fare_amount > fare_low].loc[df.fare_amount < fare_high]

In [8]:
# Transform features using the function you have defined
df_train = process_train_data(raw_train)

# Remove fields that we do not want to train with
X = df_train.drop(['key', 'fare_amount', 'pickup_datetime'], axis=1, errors='ignore')

# Extract the value you want to predict
Y = df_train['fare_amount']
print('Shape of the feature matrix: {}'.format(X.shape))

df_train.head(2)

Shape of the feature matrix: (8, 10)


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime,year,month,hour,weekday,distance
0,2010-02-03 20:51:29.0000003,12.9,2010-02-03 20:51:29+00:00,-73.954191,40.764029,-73.918043,40.766876,1,2010-02-03 20:51:29+00:00,2010,2,20,2,4.020429
1,2013-06-09 13:42:00.00000036,14.5,2013-06-09 13:42:00+00:00,-74.004507,40.741932,-74.005212,40.705272,1,2013-06-09 13:42:00+00:00,2013,6,13,6,1.126010


In [9]:
# Evaluate features with K-fold cross validation
# The higher K is, the longer it takes to run, and the higher your confidence in the score
K = 5
model = XGBRegressor(objective ='reg:squarederror')
scores = cross_val_score(model, X, Y, cv=K, scoring='neg_mean_squared_error', verbose=False)
avg_rmse = math.sqrt(abs(np.mean(scores)))

print('Average RMSE with {}-fold Cross Validation: {:.3f}'.format(K, avg_rmse))

Average RMSE with 5-fold Cross Validation: 60.085


## Evaluating Feature Importance
> After you train the model, XGBoost has a handy utility that allows you to compare the relative importance of each feature. You should use this to assess which features you created are meaningful. 

In [10]:
# Train the model again with the entire training set
model = XGBRegressor(objective ='reg:squarederror')
model.fit(X, Y)
#xgb.plot_importance(model, height=0.8)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

## Evaluating against Hidden Test Set
> Once you are satisfied with the performance of the features you have selected, you can use the trained model to make predictions on the hidden test set. **Do not change the default configuration of the model.** In task 1, you want to focus on feature selection, without worrying about tuning the model.

In [11]:
# Build final model with the entire training set
final_model = XGBRegressor(objective ='reg:squarederror')
final_model.fit(X, Y)

# Read and transform test set
raw_test = pd.read_csv('data/cc_nyc_fare_test.csv', parse_dates=['pickup_datetime'])
df_test = process_test_data(raw_test)
X_test = df_test.drop(['key', 'pickup_datetime'], axis=1, errors='ignore')

# Make predictions for test set and output a csv file
# DO NOT change the column names
df_test['predicted_fare_amount'] = final_model.predict(X_test)
df_test[['key', 'predicted_fare_amount']].to_csv('predictions.csv', index=False)

In [15]:
raw_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2010-05-21 06:28:12.0000002,2010-05-21 06:28:12+00:00,-73.948402,40.776676,-73.981739,40.759825,1
1,2012-02-07 22:39:00.000000107,2012-02-07 22:39:00+00:00,-74.012947,40.717042,-73.989977,40.757097,1
2,2012-12-04 22:22:12.0000002,2012-12-04 22:22:12+00:00,-73.977211,40.745275,-73.977211,40.745275,1
3,2011-05-08 13:15:18.0000002,2011-05-08 13:15:18+00:00,-73.947803,40.770538,-73.790332,40.643815,1
4,2015-05-20 01:40:28.0000001,2015-05-20 01:40:28+00:00,-73.971413,40.760414,-73.962608,40.762306,2


In [14]:
print(X_test.columns)
X_test.head(2)

Index(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'passenger_count', 'year', 'month', 'hour',
       'weekday', 'distance'],
      dtype='object')


,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,hour,weekday,distance
0,-73.948402,40.776676,-73.981739,40.759825,1,2010,5,6,4,3.742864
1,-74.012947,40.717042,-73.989977,40.757097,1,2012,2,22,1,2.833824
